In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [ ]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip


--2022-07-30 04:29:16--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip.1’

cats_and_dogs_filte 100%[===================>]  65.43M   278MB/s    in 0.2s    

2022-07-30 04:29:16 (278 MB/s) - ‘cats_and_dogs_filtered.zip.1’ saved [68606236/68606236]



In [ ]:
! unzip -qq /content/cats_and_dogs_filtered.zip

replace cats_and_dogs_filtered/vectorize.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
X = []
y = []

In [ ]:
import os
cat_list = os.listdir('/content/cats_and_dogs_filtered/train/cats')
dog_list = os.listdir('/content/cats_and_dogs_filtered/train/dogs')

In [ ]:
from PIL import Image
for cat in cat_list:
  im = Image.open('/content/cats_and_dogs_filtered/train/cats/'+cat)
  new_image = im.resize((128,128))
  img = np.array(new_image)
  X.append(img)
  y.append(0)

In [ ]:
for dog in dog_list:
  im = Image.open('/content/cats_and_dogs_filtered/train/dogs/'+dog)
  new_image = im.resize((128,128))
  img = np.array(new_image)
  X.append(img)
  y.append(1)

In [ ]:
X = np.array(X)
y= np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1313, shuffle=True, test_size=0.2)

In [ ]:
X_train,X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1313, shuffle=True,test_size=0.2)

In [ ]:
X_val = X_val / 255.
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
X_train.shape

(1280, 128, 128, 3)

### 전이학습

In [ ]:
vgg16 = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

vgg16.trainable=False
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:

# 튜닝

model = keras.Sequential([
    vgg16,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, 'sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15,771,713
Trainable params: 1,057,025
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
EPOCHS = 30
BATCH_SIZE = 64

model.fit(X_train, y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data = (X_val, y_val))

Epoch 1/30
20/20 [==============================] - 16s 141ms/step - loss: 0.5937 - acc: 0.7125 - val_loss: 0.4513 - val_acc: 0.7812
Epoch 2/30
20/20 [==============================] - 2s 126ms/step - loss: 0.3711 - acc: 0.8250 - val_loss: 0.3284 - val_acc: 0.8594
Epoch 3/30
20/20 [==============================] - 2s 126ms/step - loss: 0.2017 - acc: 0.9133 - val_loss: 0.2901 - val_acc: 0.8844
Epoch 4/30
20/20 [==============================] - 3s 135ms/step - loss: 0.1425 - acc: 0.9484 - val_loss: 0.2750 - val_acc: 0.8906
Epoch 5/30
20/20 [==============================] - 3s 128ms/step - loss: 0.1050 - acc: 0.9680 - val_loss: 0.2834 - val_acc: 0.8875
Epoch 6/30
20/20 [==============================] - 3s 135ms/step - loss: 0.0656 - acc: 0.9859 - val_loss: 0.2998 - val_acc: 0.8813
Epoch 7/30
20/20 [==============================] - 3s 134ms/step - loss: 0.0410 - acc: 0.9953 - val_loss: 0.2994 - val_acc: 0.8875
Epoch 8/30
20/20 [==============================] - 3s 135ms/step - loss: 0

In [ ]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 1s 55ms/step - loss: 0.8118 - acc: 0.8650


[0.8118295073509216, 0.8650000095367432]